<a href="https://colab.research.google.com/github/siting1206/2022_AIcup_NLP/blob/main/NLP_assignment2_1216.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 37.8 MB/s 
     |████████████████████████████████| 182 kB 7.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/train_ret2(ans).txt', sep='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/valid_ret2(ans).txt', sep='\t')
train_df[['snippet','question','answer']] = train_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))
valid_df[['snippet','question','answer']] = valid_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))

In [7]:
train_df.head()

,snippet,question,answer
0,"pdf arxiv org may 20 , 2016 last 8 years life ...","last 8 years life , galileo house arrest espou...",copernicus
1,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus
2,jim thorpe wikipedia james francis jim thorpe ...,2 1912 olympian football star carlisle indian ...,jim thorpe
3,1000 images jim thorpe pinterest jim o'rourke ...,2 1912 olympian football star carlisle indian ...,jim thorpe
4,fodor 's arizona grand canyon google books res...,"city yuma state record average 4 , 055 hours s...",arizona


In [8]:
valid_df.head()

,snippet,question,answer
0,ernest hemingway chronorift com moved 1921 par...,"spain 1959 , wrote dangerous summer , story ri...",hemingway
1,dangerous summer ernest hemingway librarything...,"spain 1959 , wrote dangerous summer , story ri...",hemingway
2,california google books result,valley 282 feet sea level state lowest point w...,california
3,death valley national park california home sec...,valley 282 feet sea level state lowest point w...,california
4,"chase , pnc atms dispense 1 5 bills pop across...","like banks , many grocery stores dispensing ca...",atms


In [9]:
train_df['ans_start'] = [y.index(x) if str(x) in y else 0 for x,y in zip(train_df["answer"], train_df["snippet"])]
train_df['ans_end'] = [x+len(y)-1 if str(y) in z else 0 for x,y,z in zip(train_df["ans_start"], train_df["answer"], train_df["snippet"])]

In [10]:
valid_df['ans_start'] = [y.index(x) if str(x) in y else 0 for x,y in zip(valid_df["answer"], valid_df["snippet"])]
valid_df['ans_end'] = [x+len(y)-1 if str(y) in z else 0 for x,y,z in zip(valid_df["ans_start"], valid_df["answer"], valid_df["snippet"])]

In [11]:
train_df.head()

,snippet,question,answer,ans_start,ans_end
0,"pdf arxiv org may 20 , 2016 last 8 years life ...","last 8 years life , galileo house arrest espou...",copernicus,56,66
1,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus,112,122
2,jim thorpe wikipedia james francis jim thorpe ...,2 1912 olympian football star carlisle indian ...,jim thorpe,0,10
3,1000 images jim thorpe pinterest jim o'rourke ...,2 1912 olympian football star carlisle indian ...,jim thorpe,12,22
4,fodor 's arizona grand canyon google books res...,"city yuma state record average 4 , 055 hours s...",arizona,9,16


In [12]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end
0,ernest hemingway chronorift com moved 1921 par...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,7,16
1,dangerous summer ernest hemingway librarything...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,24,33
2,california google books result,valley 282 feet sea level state lowest point w...,california,0,10
3,death valley national park california home sec...,valley 282 feet sea level state lowest point w...,california,27,37
4,"chase , pnc atms dispense 1 5 bills pop across...","like banks , many grocery stores dispensing ca...",atms,12,16


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
train_data_s = train_df['snippet'].tolist()
valid_data_s = valid_df['snippet'].tolist()

train_data_q = train_df['question'].tolist()
valid_data_q = valid_df['question'].tolist()
for i in range(0,len(train_data_q)):
  if type(train_data_q[i]) == float:
    print(i)

In [15]:
train_encodings = tokenizer(train_data_s, train_data_q, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_s, valid_data_q, truncation=True, padding=True)
train_encodings[0]

Encoding(num_tokens=167, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.decode(train_encodings['input_ids'][0])
tokenizer.decode(val_encodings['input_ids'][0])

"[CLS] ernest hemingway chronorift com moved 1921 paris, wrote articles toronto star writing newspaper, hemingway toured account spanish bullfighting, green hills africa 1935, story much time hemingway spent cuba fidel castro's 1959 revolution [SEP] spain 1959, wrote dangerous summer, story rival bullfighters [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [ ]:
train_answer = train_df[['ans_start', 'ans_end']].to_dict('records')
valid_answer = valid_df[['ans_start', 'ans_end']].to_dict('records')

In [ ]:
def add_token_positions(encodings, answers):
    ans_start, ans_end = [],[]

    for i in range(len(answers)):
        ans_start.append(encodings.char_to_token(i, answers[i]['ans_start']))
        ans_end.append(encodings.char_to_token(i, answers[i]['ans_end']))

        if ans_start[-1] is None:
            ans_start[-1] = 0
            ans_end[-1] = 0
            # continue

            
        shift = 1
        while ans_end[-1] is None:
            ans_end[-1] = encodings.char_to_token(i, answers[i]['ans_end'] - shift)
            shift += 1
            
    encodings.update({'ans_start':ans_start, 'ans_end':ans_end})

In [ ]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'ans_start', 'ans_end'])

In [ ]:
print(train_encodings['ans_start'][:10])
print(train_encodings['ans_end'][:10])

[21, 29, 1, 3, 6, 4, 1, 1, 1, 1]
[23, 31, 5, 7, 8, 6, 5, 5, 1, 1]


In [16]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [17]:
train_dataset = QADataset(train_encodings)
valid_dataset = QADataset(val_encodings)

In [18]:
next(iter(train_dataset))

{'input_ids': tensor([  101,   185,  1181,  2087,   170,  1197,  8745,  1964,  8916,  1336,
          1406,   117,  1446,  1314,   129,  1201,  1297,   117, 20003,  4759,
          1186, 16743,  4558, 14191,  1402,  6040, 13936,  5674, 19578,  1299,
           112,   188,  2749,  3701,  2507,  5117,   117,  1647,   102,  1314,
           129,  1201,  1297,   117, 20003,  4759,  1186,  1402,  6040, 13936,
          5674, 19578,  1299,   112,   188,  2749,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [19]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 2)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out

## Train

In [25]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-6)

cuda


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

In [ ]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
training_epoch = 1
loss_fct = CrossEntropyLoss()

In [ ]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            ans_start = batch['ans_start'].to(device)
            ans_end = batch['ans_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

            ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
            ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

            ans_start_loss = loss_fct(ans_start_logits, ans_start)
            ans_end_loss = loss_fct(ans_end_logits, ans_end)

            loss = ans_start_loss + ans_end_loss

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [ ]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0
    torch.save(model.state_dict(), '/bert_model')
    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        ans_start = batch['ans_start'].to(device)
        ans_end = batch['ans_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_loss = loss_fct(ans_start_logits, ans_start)
        ans_end_loss = loss_fct(ans_end_logits, ans_end)



        loss = ans_start_loss + ans_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 100 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 100))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

Epoch 0:   0%|          | 101/22740 [00:26<1:39:10,  3.80it/s, loss=9.5]

Epoch 101 Batch 100 Loss 9.8828


Epoch 0:   1%|          | 201/22740 [00:52<1:39:18,  3.78it/s, loss=7.94]

Epoch 201 Batch 200 Loss 8.5163


Epoch 0:   1%|▏         | 301/22740 [01:19<1:38:59,  3.78it/s, loss=6.21]

...

Validation Epoch 3421 Batch 3420 Loss 2.5004


 99%|█████████▉| 3452/3474 [05:25<00:02, 10.55it/s]

Validation Epoch 3451 Batch 3450 Loss 2.8397


100%|██████████| 3474/3474 [05:27<00:00, 10.62it/s]


evaluate loss:  tensor(3.9603, device='cuda:0')


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/NLP/bert_model')

In [31]:
model = myModel().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP_assignment2/bert_model'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## Predict

In [32]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    ans_sub_output = []

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_prdict = torch.argmax(ans_start_logits, 1).cpu().numpy()
        ans_end_prdict = torch.argmax(ans_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((ans_start_prdict[i].item(), ans_end_prdict[i].item()))

            ans_sub = tokenizer.decode(input_ids[i][ans_start_prdict[i]:ans_end_prdict[i]+1])
            
            ans_sub_output.append(ans_sub)
    
    return ans_sub_output, predict_pos

In [33]:
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [34]:
ans_sub_output, predict_pos = predict(valid_loader)

100%|██████████| 1737/1737 [05:19<00:00,  5.44it/s]


In [35]:
def get_output_post_fn(test, ans_sub_output):
    ans_sub = []
    for i in range(len(test)):

        ans_sub_pred = ans_sub_output[i].split()

        if ans_sub_pred is None:
            ans_sub_pred = []
        ans_sub_error_index = ans_sub_pred.index('[SEP]') if '[SEP]' in ans_sub_pred else -1

        if ans_sub_error_index != -1:
            ans_sub_pred = ans_sub_pred[:ans_sub_error_index]


        ans_sub.append(' '.join(ans_sub_pred))

    return ans_sub

In [36]:
ans_sub = get_output_post_fn(valid_df, ans_sub_output)

In [37]:
valid_df['ans_sub'] = ans_sub

In [38]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end,ans_sub
0,ernest hemingway chronorift com moved 1921 par...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,7,16,ernest hemingway
1,dangerous summer ernest hemingway librarything...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,24,33,ernest hemingway
2,california google books result,valley 282 feet sea level state lowest point w...,california,0,10,california
3,death valley national park california home sec...,valley 282 feet sea level state lowest point w...,california,27,37,california
4,"chase , pnc atms dispense 1 5 bills pop across...","like banks , many grocery stores dispensing ca...",atms,12,16,pnc atms


In [39]:
import re
import collections
import string
  
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
  
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
  
    def white_space_fix(text):
        return " ".join(text.split())
  
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
  
    def lower(text):
        return text.lower()
  
    return white_space_fix(remove_articles(remove_punc(lower(s))))
  
  
def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()
  
  
def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))
  
  
def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [48]:
em, f1 = 0,0
for i in range(valid_df.shape[0]):
  em += compute_exact(valid_df.iloc[i]["answer"], valid_df.iloc[i]['ans_sub'])
  f1 += compute_f1(valid_df.iloc[i]["answer"], valid_df.iloc[i]['ans_sub'])
em = em / len(valid_df)
f1 = f1 / len(valid_df)
print(em,f1)

0 0
0.5767292881307133 0.6733865735757599


In [42]:
test = valid_df.drop(['snippet', 'ans_start', 'ans_end'], axis=1)
test.head()

,question,answer,ans_sub
0,"spain 1959 , wrote dangerous summer , story ri...",hemingway,ernest hemingway
1,"spain 1959 , wrote dangerous summer , story ri...",hemingway,ernest hemingway
2,valley 282 feet sea level state lowest point w...,california,california
3,valley 282 feet sea level state lowest point w...,california,california
4,"like banks , many grocery stores dispensing ca...",atms,pnc atms
